In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re
import pickle
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import time, datetime

tqdm.pandas()

def timestamp():
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S:')
    return st


DIRECTORY = 'E:/Scrapped-Data/TED-Talks'
print(timestamp(), 'Libraries Imported')

Using TensorFlow backend.


21:45:27: Libraries Imported


In [2]:
NOT_WORDS_REGEX = re.compile(r"[^a-zA-Z]")
EXTRAWHITESPACES_REGEX = re.compile(r"\s+")

data = pd.read_csv(DIRECTORY+'/transcripts.csv')['transcript'].progress_apply(lambda x : EXTRAWHITESPACES_REGEX.sub(' ',
                                                                                NOT_WORDS_REGEX.sub(' ', x))).values

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
print(timestamp(), 'Tokenizer Trained')

count_thres = 5
low_count_words = [w for w,c in tqdm(tokenizer.word_counts.items()) if c < count_thres]
for w in tqdm(low_count_words):
    del tokenizer.word_index[w]
    del tokenizer.word_docs[w]
    del tokenizer.word_counts[w]
print(timestamp(), 'Tokenizer Updated')

pickle.dump(tokenizer, open(DIRECTORY+'/Tokenizer.pkl', 'wb'))
print(timestamp(), 'Tokenizer Pickled')

21:45:38: Tokenizer Trained



21:45:38: Tokenizer Updated
21:45:38: Tokenizer Pickled


In [4]:
sorted(tokenizer.word_counts.items(), key=lambda x: x[1])

[('retiring', 5),
 ('discotheque', 5),
 ('compartments', 5),
 ('lynne', 5),
 ('reconstitute', 5),
 ('sequestration', 5),
 ('offsets', 5),
 ('arcane', 5),
 ('opposable', 5),
 ('woe', 5),
 ('equivalents', 5),
 ('hawkins', 5),
 ('gunned', 5),
 ('antiquated', 5),
 ('seeding', 5),
 ('expressway', 5),
 ('roofing', 5),
 ('lovins', 5),
 ('bogota', 5),
 ('outlawed', 5),
 ('walkways', 5),
 ('karolinska', 5),
 ('testicular', 5),
 ('tilts', 5),
 ('horoscope', 5),
 ('libra', 5),
 ('lamanites', 5),
 ('impregnated', 5),
 ('condescending', 5),
 ('concludes', 5),
 ('whim', 5),
 ('intermission', 5),
 ('steward', 5),
 ('landmass', 5),
 ('induces', 5),
 ('appendix', 5),
 ('endorsing', 5),
 ('pluralistic', 5),
 ('notoriety', 5),
 ('psalm', 5),
 ('meetup', 5),
 ('natal', 5),
 ('unplanned', 5),
 ('trousers', 5),
 ('filipino', 5),
 ('stint', 5),
 ('mouthpiece', 5),
 ('headmaster', 5),
 ('luckiest', 5),
 ('brin', 5),
 ('scorecard', 5),
 ('epidemiological', 5),
 ('antiviral', 5),
 ('litany', 5),
 ('birthed', 5)

In [5]:
tokenizer = pickle.load(open(DIRECTORY+'/Tokenizer.pkl', 'rb'))

data = tokenizer.texts_to_sequences(tqdm(data))
print(timestamp(), 'Data Tokenized')

data = [item for sublist in tqdm(data) for item in sublist]
print(timestamp(), 'Token List Flattened', len(data))

print(len(tokenizer.word_counts))


21:45:42: Data Tokenized



21:45:43: Token List Flattened 5145559
23131


In [6]:
train_len = 3 + 1
sequences = []
for i in tqdm(range(train_len, len(data)+1)):
    seq = data[i-train_len:i]
    sequences.append(seq)
sequences = np.array(sequences)
print(timestamp(), 'Sequenced', len(sequences))


21:45:55: Sequenced 5145556


In [7]:
x = sequences[:, :-1]
y = sequences[:, -1]
vocab_size = len(tokenizer.word_counts)
y = to_categorical(y, num_classes=vocab_size+1)

MemoryError: 